# StudentPerformancePredictionML
### This project understands how the student's performance (test scores) is affected by other variables such as Gender, Ethnicity, Parental level of education, Lunch and Test preparation course.

*******************************
*******************************
<br>

## Main Folder 👇

-----------------------
<br>

### setup.py 👇
(/StudentPerformancePredictionML/setup.py)

The setup.py is responsible for creating my machine learning project as a package

```python
setup(
name='StudentPerformancePredictionML',   # Name of project
version='0.0.1',   # Version 
author='Ninad',    # Author name
author_email='ninad.karlekar@vsit.edu.in',  # Author email-id
packages=find_packages(),
install_requires=get_requirements('requirements.txt')
)
# When you use `find_packages()` in your setup.py file, it will search the current directory and its subdirectories for any packages that contain an __init__.py file,
```

-----------------------
<br>

### requirements.txt 
(/StudentPerformancePredictionML/requirements.txt)

`requirements.txt` file will have all packages that we will use while implimenting project